In [1]:
from NewsCrawler import NewsCrawler

In [2]:
NewsCrawler.navercrawl('https://n.news.naver.com/article/015/0004837163')

('200만원 항공권 취소했더니… 청천벽력 같은 답변 돌아왔다',
 '해외여행 늘면서 관련 소비자 상담 급증 해외 OTA 키위닷컴 환불불가 피해주의보 # 올해 9월 괌 여행을 계획 중인 A씨는 지난달 가격비교 사이트를 이용해 찾은 해외 온라인 여행사(OTA) 키위닷컴 에서 인천~괌 왕복 항공권 2매를 196만원에 구입했다. 다음날 개인 사정으로 구매를 취소하려 했으나 키위닷컴은 특정 기간 내 사용 가능한 적립금으로 10유로(약 1만5000원)만 지급했다.A씨가 나머지 금액 환불을 문의하자 키위닷컴은 상품 판매 페이지와 약관에 환불 불가 를 사전 안내한 점을 들어 추가 환불이 불가능하다고 답했다.이처럼 여행 관련 소비자 피해 사례가 잇따르고 있다. 엔데믹(감염병의 풍토병 전환)과 함께 해외여행 수요가 폭증한 가운데 신종 코로나바이러스 감염증(코로나19) 사태를 거치며 커진 여행비용 부담을 줄이려다 오히려 피해를 본 셈이다. 전문가들은 항공권과 숙박 등예약 시 거래 조건을 꼼꼼하게 확인해야 한다고 당부했다. OTA 키위닷컴 주의보…소비자 상담 급증 올해 들어 해외 OTA 키위닷컴 관련 소비자 상담이 크게 늘면서 한국소비자원은 주의를 당부했다. 25일 소비자원에 따르면 올해 1분기 공정거래위원회가 운영하는 ‘1372소비자상담센터’에 접수된 키위닷컴 관련 피해 사례는 95건으로 지난해 1분기(7건)의 10배가 넘었다. 지난해 4분기(46건)보다도 급증했다.올해 1분기 접수된 상담 95건 대부분이 A씨와 같이 ‘취소·환불·교환 지연 및 거부’(89건·93.8%) 사례였다. 키위닷컴이 판매 페이지에 ‘자발적 취소 시 환불 불가’ 조건을 표기했다는 점을 들어 환불을 거부했고, 적립금 10유로만 지급됐다는 내용이 골자다.해당 약관에 따르면 소비자는 개별 항공권 환불 규정에 의해 환불받을 수 있는 금액인 전액 또는 취소수수료 공제 후 잔액이 아닌 10유로 적립금만 돌려받게 돼 있다. 소비자원은 약관에는 소비자가 10유로의 적립금 지급을 요구하지 않고, 직접 항공사에 취소·환불

In [3]:
from bs4 import BeautifulSoup
import requests
import re

In [4]:
def textProcessing( text: str) -> str:
    text = text.replace(u'\n', u' ')
    text = text.replace(u'\'', u' ')
    text = text.replace(u'\\\'', u' ')
    text = text.replace(u'[', u'')
    text = text.replace(u']', u'')
    text = text.replace(u'\xa0', u' ')
    text = text.replace(u'\"', u'')
    text = text.replace(u'\'', u'')
    text = text.replace(u'  ', u' ')
    text = text.replace(u'  ', u' ')
    text = text.replace(u'  ', u' ')

    return text.strip()


naver_image_caption_regex = r'<em[ㄱ-ㅎ가-힣a-zA-Z0-9\s.,%\'\"()·“‘’”=_>/]+</em>'
naver_image_caption = re.compile(naver_image_caption_regex)
naver_news_press_regex = r'alt=\"(?P<extract>[ㄱ-ㅎ가-힣a-zA-Z0-9]+)\"'
naver_news_press = re.compile(naver_news_press_regex)

In [5]:
def navercrawl(url: str) -> str:
    '''
    url검사 x
    naver 뉴스일때 크롤링 하는 함수
    '''
    # 이거 안붙이면 nave가 봇으로 인식
    headers = {'User-Agent': 'Mozilla/5.0'}
    web_page = requests.get(url, headers=headers)
    if web_page.status_code != 200:
        return 'error : can\'t get html'
    soup = BeautifulSoup(web_page.content, 'html.parser')
    html = str(soup.select('#dic_area'))
    title = str(soup.select('#title_area > span'))

    article_date = str(soup.select('.media_end_head_info_datestamp'))
    article_date = article_date.replace(u'[\n\n입력', u'')
    article_date = article_date.replace(u'\n\n기사원문\n]', u'')
    article_date = BeautifulSoup(article_date, "lxml").text

    press = str(soup.select('img.media_end_head_top_logo_img:nth-child(1)'))
    press = naver_news_press.search(press)
    press = press.group(1)

    reporter = str(soup.select(
        '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_journalist > a > em'))
    reporter = BeautifulSoup(reporter, "lxml").text
    reporter = textProcessing(reporter)

    section = str(soup.select('#contents > div.media_end_categorize > a > em'))
    section = BeautifulSoup(section, "lxml").text
    section = section.replace('[', '')
    section = section.replace(']', '')

    # 구처리

    # news_data = cls.naver_crawling_pattern.findall(html)
    # news_string =''
    # for data in news_data:
    # news_string += data[1]
    # news_string = news_string.replace(u'\xa0', u' ')
    # return news_string
    # print(url)
    html = naver_image_caption.sub('', str(html))
    cleantext = BeautifulSoup(html, "lxml").text
    title = BeautifulSoup(title, "lxml").text
    title = textProcessing(title)
    cleantext = textProcessing(cleantext)
    return title, press, reporter, cleantext, section, article_date, 'ㅇㅇ'

In [6]:
_,_,_,_,a,b,c =navercrawl('https://n.news.naver.com/article/015/0004837163')

In [7]:
article_date_regex = r'입력(?P<in>[0-9]+\.[0-9]+\.[0-9]+\.\s[오전|오후]+\s[0-9]:[0-9]+)\n+수정(?P<modify>[0-9]+\.[0-9]+\.[0-9]+\.\s[오전|오후]+\s[0-9]:[0-9]+)'

In [8]:
only_article_date_regex = r'입력(?P<in>[0-9]+\.[0-9]+\.[0-9]+\.\s[오전|오후]+\s[0-9]:[0-9]+)'

In [9]:
aa = re.compile(article_date_regex)

In [10]:
aaa = re.compile(only_article_date_regex)

In [11]:
aaa.search(bbb).groups()[0]

NameError: name 'bbb' is not defined

In [ ]:
c = '[\n\n입력2023.04.25. 오후 8:02\n\n\n\n\n기사원문\n]'

In [ ]:
_,_,_,_,a,bbb,c =navercrawl('https://n.news.naver.com/article/022/0003806515')

In [ ]:
bbb

In [ ]:
aa.search(bbb)

In [ ]:
test = aa.search(b)

In [ ]:
test2 = aa.search(c)

In [ ]:
test2.groups()

In [ ]:
test.groups()

In [ ]:
from datetime import datetime,timedelta

In [ ]:
test1 =test1.replace('오후','')

In [ ]:
bbb

In [ ]:
bbb.find("기사")

In [ ]:
if bbb.find('수정') != -1:
    print('찾음')

In [ ]:
date_time_obj = datetime.strptime(test1,'%Y.%m.%d. %H:%M')

In [ ]:
date_time_obj

In [ ]:
date_time_obj + timedelta(hours=12)

In [ ]:
date_time_obj.strftime('%Y-%m-%d %H:%M:%S')


In [ ]:
test1

In [ ]:
def transform_datetime(string : str ) -> list:
    '''

    :param string:
    :return: datetime tuple
    '''

    article_date_regex = r'입력(?P<in>[0-9]+\.[0-9]+\.[0-9]+\.\s[오전|오후]+\s[0-9]:[0-9]+)\n+수정(?P<modify>[0-9]+\.[0-9]+\.[0-9]+\.\s[오전|오후]+\s[0-9]:[0-9]+)'
    only_article_date_regex = r'입력(?P<in>[0-9]+\.[0-9]+\.[0-9]+\.\s[오전|오후]+\s[0-9]:[0-9]+)'


    if string.find('수정') != -1 :
        regex = re.compile(article_date_regex)
    else:
        regex = re.compile(only_article_date_regex)

    temp_tuple =  regex.search(string).groups()
    result_list = []
    for date_str in temp_tuple:

        if date_str.find('오후') == -1:
            date_str =date_str.replace('오전','')
            date_time_obj = datetime.strptime(date_str,'%Y.%m.%d. %H:%M')
        else:
            date_str =date_str.replace('오후','')
            date_time_obj = datetime.strptime(date_str,'%Y.%m.%d. %H:%M')
            date_time_obj + timedelta(hours=12)
        result_list.append(date_time_obj)
    return result_list










In [ ]:
transform_datetime(bbb)

In [1]:
from NewsCrawler import  NewsCrawler

In [2]:
NewsCrawler.transform_datetimestring()

TypeError: transform_datetimestring() missing 1 required positional argument: 'string'

In [36]:
a,b,c,d,e,f,g =NewsCrawler.navercrawl('https://n.news.naver.com/article/277/0005251916?ntype=RANKING')

In [39]:
g

'2023-04-28 07:29:00'

In [24]:
import re

In [25]:
regex = r'(http|https)://n.news.naver.com/article/\d+/\d+'

In [26]:
a = re.compile(regex)

In [32]:
b = a.match('https://n.news.naver.com/article/277/0005251916?ntype=RANKING')

In [33]:
if b :
    print('실패')
print('실패아님')

실패
실패아님


In [35]:
b[0]

'https://n.news.naver.com/article/277/0005251916'

In [31]:
if not b:
    print('실패')

실패
